In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medical-segmentation-decathlon-lung/._labelsTr
/kaggle/input/medical-segmentation-decathlon-lung/._imagesTr
/kaggle/input/medical-segmentation-decathlon-lung/._imagesTs
/kaggle/input/medical-segmentation-decathlon-lung/._dataset.json
/kaggle/input/medical-segmentation-decathlon-lung/dataset.json
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_040.nii/lung_040.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_052.nii/lung_052.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_088.nii/lung_088.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_067.nii/lung_067.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_089.nii/lung_089.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_068.nii/lung_068.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_056.nii/lung_056.nii
/kaggle/input/medical-segmentation-decathlon-lung/imagesTs/lung_087.nii/lung_087.nii
/kaggle/i

In [2]:
!apt-get update && apt-get install -y tree
!tree /kaggle/input/

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,825 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease

In [3]:
# Install required packages
%pip install ultralytics nibabel scikit-image opencv-python-headless tqdm

import os
import glob
import numpy as np
import nibabel as nib
import cv2
from tqdm import tqdm
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import yaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [4]:
# --- Define Input Paths ---
DATA_DIR = "/kaggle/input/medical-segmentation-decathlon-lung/"
IMAGE_DIR = os.path.join(DATA_DIR, "imagesTr")
LABEL_DIR = os.path.join(DATA_DIR, "labelsTr")

# Find all the NIfTI files using the nested pattern
# This uses the same glob pattern that worked for you before
image_paths = sorted(glob.glob(os.path.join(IMAGE_DIR, "*/*.nii")))
label_paths = sorted(glob.glob(os.path.join(LABEL_DIR, "*/*.nii")))

# --- Define Output Paths for YOLO Dataset ---
# We'll create our processed dataset in the /kaggle/working/ directory
YOLO_DATA_DIR = "/kaggle/working/lung_dataset_yolo/"
os.makedirs(YOLO_DATA_DIR, exist_ok=True)
for sub in ['train', 'val']:
    os.makedirs(os.path.join(YOLO_DATA_DIR, 'images', sub), exist_ok=True)
    os.makedirs(os.path.join(YOLO_DATA_DIR, 'labels', sub), exist_ok=True)

print("YOLO directory structure created successfully.")

YOLO directory structure created successfully.


In [5]:
# Cell 3 (Corrected Version)

def convert_nii_to_yolo(image_path, label_path, output_img_dir, output_lbl_dir, patient_id):
    """
    Converts a single 3D NIfTI scan and its label into 2D slices in YOLO format.
    """
    img_nii = nib.load(image_path)
    lbl_nii = nib.load(label_path)
    image_data = img_nii.get_fdata()
    label_data = lbl_nii.get_fdata()

    for i in range(image_data.shape[2]):
        slice_label = label_data[:, :, i]

        if np.sum(slice_label) > 0:
            slice_image = image_data[:, :, i]
            slice_image_normalized = cv2.normalize(slice_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            slice_image_rgb = cv2.cvtColor(slice_image_normalized, cv2.COLOR_GRAY2BGR)
            img_filename = f"{patient_id}_slice_{i}.png"
            cv2.imwrite(os.path.join(output_img_dir, img_filename), slice_image_rgb)

            contours, _ = cv2.findContours(slice_label.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            label_filename = f"{patient_id}_slice_{i}.txt"
            h, w = slice_label.shape

            with open(os.path.join(output_lbl_dir, label_filename), 'w') as f:
                for contour in contours:
                    # --- THIS IS THE FIX ---
                    # Only write polygons with 3 or more points
                    if contour.shape[0] >= 3:
                        f.write("0")
                        points = contour.flatten()
                        for j in range(0, len(points), 2):
                            x = points[j] / w
                            y = points[j+1] / h
                            f.write(f" {x:.6f} {y:.6f}")
                        f.write("\n")

print("Conversion function is ready.")

Conversion function is ready.


In [6]:
# Split the original NIfTI files into training and validation sets
train_img_paths, val_img_paths, train_lbl_paths, val_lbl_paths = train_test_split(
    image_paths, label_paths, test_size=0.2, random_state=42
)

# Process the training set
print("Processing training data...")
for img_p, lbl_p in tqdm(zip(train_img_paths, train_lbl_paths), total=len(train_img_paths)):
    patient_id = os.path.basename(os.path.dirname(img_p)) # e.g., 'lung_001.nii'
    convert_nii_to_yolo(img_p, lbl_p, 
                        os.path.join(YOLO_DATA_DIR, 'images', 'train'), 
                        os.path.join(YOLO_DATA_DIR, 'labels', 'train'), 
                        patient_id)

# Process the validation set
print("\nProcessing validation data...")
for img_p, lbl_p in tqdm(zip(val_img_paths, val_lbl_paths), total=len(val_img_paths)):
    patient_id = os.path.basename(os.path.dirname(img_p))
    convert_nii_to_yolo(img_p, lbl_p, 
                        os.path.join(YOLO_DATA_DIR, 'images', 'val'), 
                        os.path.join(YOLO_DATA_DIR, 'labels', 'val'), 
                        patient_id)

print("\nData conversion complete!")

Processing training data...


100%|██████████| 50/50 [04:54<00:00,  5.89s/it]



Processing validation data...


100%|██████████| 13/13 [01:11<00:00,  5.51s/it]


Data conversion complete!


In [7]:
# Create the YAML configuration file
data_yaml = {
    'train': '../lung_dataset_yolo/images/train',
    'val': '../lung_dataset_yolo/images/val',
    'nc': 1,  # Number of classes
    'names': ['tumor']  # Class names
}

with open(os.path.join(YOLO_DATA_DIR, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)

print("data.yaml file created successfully:")
!cat /kaggle/working/lung_dataset_yolo/data.yaml

data.yaml file created successfully:
names:
- tumor
nc: 1
train: ../lung_dataset_yolo/images/train
val: ../lung_dataset_yolo/images/val


In [8]:
# Load a pretrained YOLOv8 segmentation model
model = YOLO('yolov8n-seg.pt')

# Train the model
results = model.train(
    data=os.path.join(YOLO_DATA_DIR, 'data.yaml'),
    epochs=50,      # Number of training epochs
    imgsz=512,      # Image size for training
    batch=16,       # Batch size
    name='yolov8_lung_seg' # Name for the training run
)

Ultralytics 8.3.226 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/lung_dataset_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_lung_seg7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        396        418      0.662      0.506      0.509      0.212      0.711      0.464       0.48      0.186
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/runs/segment/yolov8_lung_seg7


In [9]:
# The training process automatically runs validation.
# You can see metrics like mAP50-95 in the training output.
# The best model weights are saved automatically.

# Let's find the path to the best model
best_model_path = '/kaggle/working/runs/segment/yolov8_lung_seg2/weights/best.pt'

# Load the best model
model = YOLO(best_model_path)

# Run validation again explicitly (optional)
metrics = model.val()

# --- THIS IS THE CORRECTED LINE ---
# We use metrics.seg instead of metrics.mask
print(f"Validation Metrics (Mask mAP50): {metrics.seg.map50}")

# You can now find all your training results, graphs, and validation images
# in the directory: /kaggle/working/runs/segment/yolov8_lung_seg2

Ultralytics 8.3.226 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8x-seg summary (fused): 125 layers, 71,721,619 parameters, 0 gradients, 327.9 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3350.2±522.7 MB/s, size: 333.8 KB)
val: Scanning /kaggle/working/lung_dataset_yolo/labels/val.cache... 396 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 396/396 759.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 1.1it/s 23.7s1.0ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        396        418      0.733      0.539      0.534      0.209       0.73      0.537      0.511      0.199
Speed: 1.1ms preprocess, 54.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /kaggle/working/runs/segment/val7
Validation Metrics (Mask mAP50): 0.5111666810464688


In [10]:
import cv2
import torch
import nibabel as nib
from tqdm import tqdm
import numpy as np
from ultralytics import YOLO
from IPython.display import HTML, display
import random

# --- 1. Load your best trained model ---
best_model_path = '/kaggle/working/runs/segment/yolov8_lung_seg2/weights/best.pt'
model = YOLO(best_model_path)

# --- 2. Select a random sample from your validation set ---
# (Using the val paths defined in Cell 4 of the original notebook)
# sample_idx = random.randint(0, len(val_img_paths) - 1
sample_idx = 11
nii_image_path = val_img_paths[sample_idx]
nii_label_path = val_lbl_paths[sample_idx]

print(f"Visualizing sample: {os.path.basename(os.path.dirname(nii_image_path))}")

# --- 3. Load the 3D NIfTI image and label ---
img_nii = nib.load(nii_image_path)
lbl_nii = nib.load(nii_label_path)
image_data = img_nii.get_fdata()
label_data = lbl_nii.get_fdata()

# --- 4. Setup Video Writer ---
output_video_path = '/kaggle/working/prediction_visualization.mp4'
h, w, num_slices = image_data.shape
# The final video frame will be 3 times as wide
frame_size = (w * 3, h) 
fps = 10
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

# --- 5. Process each slice and create the video ---
# The tqdm wrapper creates the progress bar
for i in tqdm(range(num_slices), desc="Creating video..."):
    # Get the original slice and normalize it to 0-255
    original_slice = image_data[:, :, i]
    slice_normalized = cv2.normalize(original_slice, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # Convert to 3-channel BGR for visualization and prediction
    slice_bgr = cv2.cvtColor(slice_normalized, cv2.COLOR_GRAY2BGR)

    # Get the ground truth mask for this slice
    gt_mask = label_data[:, :, i].astype(np.uint8)

    # --- Run Prediction ---
    results = model.predict(slice_bgr, verbose=False)
    
    # --- Create the 3 visualization panels ---
    # Panel 1: Original Image
    panel_original = slice_bgr.copy()
    cv2.putText(panel_original, "Original Slice", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Panel 2: Ground Truth Overlay
    panel_gt = slice_bgr.copy()
    gt_contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(panel_gt, gt_contours, -1, (0, 255, 0), 2) # Green for Ground Truth
    cv2.putText(panel_gt, "Ground Truth", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Panel 3: Prediction Overlay
    panel_pred = slice_bgr.copy()
    cv2.putText(panel_pred, "YOLO Prediction", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # Check if the model made any predictions on this slice
    if results[0].masks is not None:
        pred_mask_tensor = results[0].masks.data[0]
        pred_mask = pred_mask_tensor.cpu().numpy().astype(np.uint8)
        pred_contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(panel_pred, pred_contours, -1, (0, 0, 255), 2) # Red for Prediction

    # --- Combine panels and write to video ---
    combined_frame = np.hstack((panel_original, panel_gt, panel_pred))
    video_writer.write(combined_frame)

# Release the video writer
video_writer.release()
print(f"\nVideo saved successfully to {output_video_path}")

# --- 6. Display the video in the notebook ---
# This creates an HTML5 video player to show the result
video_html = f"""
<video width="1024" controls>
  <source src="{output_video_path}" type="video/mp4">
</video>
"""
display(HTML(video_html))

Visualizing sample: lung_016.nii


Creating video...: 100%|██████████| 228/228 [00:13<00:00, 16.33it/s]


Video saved successfully to /kaggle/working/prediction_visualization.mp4


In [11]:
import cv2
import torch
import nibabel as nib
from tqdm import tqdm
import numpy as np
from ultralytics import YOLO
from IPython.display import HTML, display
import random
import os
import glob

# --- 1. Load your best trained model ---
best_model_path = '/kaggle/working/runs/segment/yolov8_lung_seg2/weights/best.pt'
model = YOLO(best_model_path)

# --- 2. Get paths for the test data ---
TEST_IMAGE_DIR = os.path.join(DATA_DIR, "imagesTs") # DATA_DIR was defined in Cell 2
test_image_paths = sorted(glob.glob(os.path.join(TEST_IMAGE_DIR, "*/*.nii")))

# --- 3. Select a random sample from your test set ---
sample_idx = random.randint(0, len(test_image_paths) - 1)
nii_test_image_path = test_image_paths[sample_idx]

print(f"Visualizing test sample: {os.path.basename(os.path.dirname(nii_test_image_path))}")

# --- 4. Load the 3D NIfTI image ---
img_nii = nib.load(nii_test_image_path)
image_data = img_nii.get_fdata()

# --- 5. Setup Video Writer ---
output_test_video_path = '/kaggle/working/test_prediction_visualization.mp4'
h, w, num_slices = image_data.shape
# The video frame will be 2 times as wide (Original + Prediction)
frame_size = (w * 2, h) 
fps = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
test_video_writer = cv2.VideoWriter(output_test_video_path, fourcc, fps, frame_size)

# --- 6. Process each slice and create the video ---
for i in tqdm(range(num_slices), desc="Creating test set video..."):
    # Get the original slice and normalize it to 0-255
    original_slice = image_data[:, :, i]
    slice_normalized = cv2.normalize(original_slice, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # Convert to 3-channel BGR for visualization and prediction
    slice_bgr = cv2.cvtColor(slice_normalized, cv2.COLOR_GRAY2BGR)

    # --- Run Prediction ---
    results = model.predict(slice_bgr, verbose=False)
    
    # --- Create the 2 visualization panels ---
    # Panel 1: Original Image
    panel_original = slice_bgr.copy()
    cv2.putText(panel_original, "Original Test Slice", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Panel 2: Prediction Overlay
    panel_pred = slice_bgr.copy()
    cv2.putText(panel_pred, "YOLO Prediction", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    if results[0].masks is not None:
        pred_mask_tensor = results[0].masks.data[0]
        pred_mask = pred_mask_tensor.cpu().numpy().astype(np.uint8)
        pred_contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(panel_pred, pred_contours, -1, (0, 0, 255), 2) # Red for Prediction

    # --- Combine panels and write to video ---
    combined_frame = np.hstack((panel_original, panel_pred))
    test_video_writer.write(combined_frame)

# Release the video writer
test_video_writer.release()
print(f"\nTest set video saved successfully to {output_test_video_path}")

# --- 7. Display the video in the notebook ---
test_video_html = f"""
<video width="1024" controls>
  <source src="{output_test_video_path}" type="video/mp4">
</video>
"""
display(HTML(test_video_html))

Visualizing test sample: lung_017.nii


Creating test set video...: 100%|██████████| 580/580 [00:32<00:00, 17.72it/s]


Test set video saved successfully to /kaggle/working/test_prediction_visualization.mp4


**# ANALYZER**


In [ ]:
import nibabel as nib
import numpy as np
import cv2
import os
from tqdm import tqdm
from ultralytics import YOLO

print("--- Step 1: Creating Stage 2 'Analyzer' Dataset ---")

# --- 1. Load your trained YOLO model ---
# Make sure this path is correct for your notebook
best_model_path = '/kaggle/working/runs/segment/yolov8_lung_seg2/weights/best.pt' 
model = YOLO(best_model_path)

# --- 2. Define Output Dirs for the new dataset ---
STAGE2_DATA_DIR = "/kaggle/working/stage_2_analyzer_data/"
CLASS_0_HEALTHY_DIR = os.path.join(STAGE2_DATA_DIR, "class_0_healthy") # False Positives
CLASS_1_CANCER_DIR = os.path.join(STAGE2_DATA_DIR, "class_1_cancer")   # Real Tumors
os.makedirs(CLASS_0_HEALTHY_DIR, exist_ok=True)
os.makedirs(CLASS_1_CANCER_DIR, exist_ok=True)

# --- 3. Get all your patient scans (from your original script) ---
# This assumes 'train_img_paths', 'val_img_paths', 'train_lbl_paths', 
# and 'val_lbl_paths' are still in memory from your previous cells.
all_image_paths = train_img_paths + val_img_paths
all_label_paths = train_lbl_paths + val_lbl_paths

print(f"Processing {len(all_image_paths)} patient scans to build Stage 2 dataset...")

# --- 4. Main Loop: Find Real Tumors and False Positives ---
patch_id = 0
for img_p, lbl_p in tqdm(zip(all_image_paths, all_label_paths), total=len(all_image_paths)):
    
    img_nii = nib.load(img_p)
    lbl_nii = nib.load(lbl_p)
    image_data = img_nii.get_fdata()
    label_data = lbl_nii.get_fdata()

    patient_id = os.path.basename(os.path.dirname(img_p))

    for i in range(image_data.shape[2]): # Loop through each 2D slice
        slice_label = label_data[:, :, i].astype(np.uint8)
        slice_image = image_data[:, :, i]
        
        # Normalize and convert for prediction
        slice_normalized = cv2.normalize(slice_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        slice_bgr = cv2.cvtColor(slice_normalized, cv2.COLOR_GRAY2BGR) # YOLO needs 3 channels

        # --- A: Find and Save REAL Tumors (Ground Truth) ---
        # This gives our model the "Cancer" examples
        if np.sum(slice_label) > 0: # If there's a real tumor in this slice
            contours, _ = cv2.findContours(slice_label, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                # Crop the patch from the original (normalized) image
                patch = slice_normalized[y:y+h, x:x+w]
                if patch.size > 0:
                    patch_filename = f"{patient_id}_slice_{i}_gt_{patch_id}.png"
                    cv2.imwrite(os.path.join(CLASS_1_CANCER_DIR, patch_filename), patch)
                    patch_id += 1

        # --- B: Find and Save FALSE Positives (YOLO's Mistakes) ---
        # This gives our model the "Healthy" (but tricky) examples
        results = model.predict(slice_bgr, verbose=False)
        
        if results[0].masks is not None:
            # Loop over each mask YOLO found
            for mask_tensor in results[0].masks.data:
                pred_mask = mask_tensor.cpu().numpy().astype(np.uint8)
                
                # Check if this prediction overlaps with the REAL tumor
                # We add the masks: 0=nothing, 1=GT, 2=Pred, 3=Overlap
                combined_mask = slice_label + (pred_mask * 2)
                overlap_area = np.sum(combined_mask == 3)
                
                # If there's NO overlap, it's a False Positive
                if overlap_area == 0:
                    contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    for contour in contours:
                        x, y, w, h = cv2.boundingRect(contour)
                        # Crop the patch from the original (normalized) image
                        patch = slice_normalized[y:y+h, x:x+w]
                        if patch.size > 0:
                            patch_filename = f"{patient_id}_slice_{i}_fp_{patch_id}.png"
                            cv2.imwrite(os.path.join(CLASS_0_HEALTHY_DIR, patch_filename), patch)
                            patch_id += 1

print(f"\n--- Stage 2 Dataset Creation Complete ---")
print(f"Found {len(os.listdir(CLASS_1_CANCER_DIR))} real tumor patches.")
print(f"Found {len(os.listdir(CLASS_0_HEALTHY_DIR))} false positive patches.")

--- Step 1: Creating Stage 2 'Analyzer' Dataset ---
Processing 63 patient scans to build Stage 2 dataset...


 49%|████▉     | 31/63 [10:24<09:01, 16.93s/it]

In [ ]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import cv2
import os # We'll need os for the test at the end

print("--- Step 2: ECE Feature Extractor Function ---")

def get_ece_features(image_patch):
    """
    Takes a single (grayscale) image patch and returns
    its ECE texture features based on GLCM.
    
    This works by measuring the spatial relationship of pixels,
    quantifying texture with metrics from information theory.
    """
    
    # Ensure patch is 8-bit (skimage needs this)
    if image_patch.dtype != np.uint8:
        # Check if the image is already normalized (0-1)
        if image_patch.max() <= 1.0:
            image_patch = (image_patch * 255).astype(np.uint8)
        else:
            image_patch = cv2.normalize(image_patch, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # 1. Calculate the Gray-Level Co-occurrence Matrix (GLCM)
    # We check 4 directions (0, 45, 90, 135 deg) and average them
    glcm = graycomatrix(image_patch, 
                        distances=[5], # The pixel distance to check
                        angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], # The angles
                        levels=256,
                        symmetric=True, 
                        normed=True)

    # 2. Extract the texture properties
    # These are your ECE features!
    contrast = np.mean(graycoprops(glcm, 'contrast'))
    dissimilarity = np.mean(graycoprops(glcm, 'dissimilarity'))
    homogeneity = np.mean(graycoprops(glcm, 'homogeneity'))
    energy = np.mean(graycoprops(glcm, 'energy'))
    correlation = np.mean(graycoprops(glcm, 'correlation'))
    
    # Return the ECE feature vector
    return np.array([contrast, dissimilarity, homogeneity, energy, correlation])

# --- Test the function ---
# (Assuming the variables from Step 1 are still in memory)
try:
    if len(os.listdir(CLASS_1_CANCER_DIR)) > 0:
        sample_patch_path = os.path.join(CLASS_1_CANCER_DIR, os.listdir(CLASS_1_CANCER_DIR)[0])
        sample_patch = cv2.imread(sample_patch_path, cv2.IMREAD_GRAYSCALE)
        
        if sample_patch is not None:
            features = get_ece_features(sample_patch)
            print(f"Extracted {len(features)} ECE features from a sample patch.")
            print(f"Features: [Contrast, Dissimilarity, Homogeneity, Energy, Correlation]")
            print(f"Values: {features}")
        else:
            print("Could not read sample patch for testing.")
    else:
        print("No sample patches found in CLASS_1_CANCER_DIR to test.")
except NameError:
    print("Run this cell again after running Step 1 to test the function.")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import cv2
import glob
import os

print("--- Step 3 (Pure PyTorch): Building the Hybrid ECE-CNN Analyzer ---")

# --- 1. Load all patch data and labels (Same as before) ---
print("Loading all patches and calculating ECE features...")
all_image_patches = []
all_ece_features = []
all_labels = []

CLASS_0_HEALTHY_DIR = os.path.join(STAGE2_DATA_DIR, "class_0_healthy")
CLASS_1_CANCER_DIR = os.path.join(STAGE2_DATA_DIR, "class_1_cancer")

for img_path in tqdm(glob.glob(os.path.join(CLASS_0_HEALTHY_DIR, "*.png"))):
    patch = cv2.imread(img_path)
    if patch is not None:
        patch_resized = cv2.resize(patch, (128, 128))
        patch_gray = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2GRAY)
        all_image_patches.append(patch_resized)
        all_ece_features.append(get_ece_features(patch_gray))
        all_labels.append(0)

for img_path in tqdm(glob.glob(os.path.join(CLASS_1_CANCER_DIR, "*.png"))):
    patch = cv2.imread(img_path)
    if patch is not None:
        patch_resized = cv2.resize(patch, (128, 128))
        patch_gray = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2GRAY)
        all_image_patches.append(patch_resized)
        all_ece_features.append(get_ece_features(patch_gray))
        all_labels.append(1)

# --- 2. Convert to Tensors (PyTorch Format) ---
X_images_np = np.array(all_image_patches)
X_ece_np = np.array(all_ece_features)
y_np = np.array(all_labels)

# PyTorch needs (N, C, H, W) format, so we transpose
X_images_np = np.transpose(X_images_np, (0, 3, 1, 2))

# Convert to float32 Tensors, which PyTorch expects
X_images_torch = torch.tensor(X_images_np, dtype=torch.float32) / 255.0 # Normalize
X_ece_torch = torch.tensor(X_ece_np, dtype=torch.float32)
y_torch = torch.tensor(y_np, dtype=torch.float32)

print(f"\nTotal samples: {len(y_torch)}")
print(f"Image tensor shape: {X_images_torch.shape}")
print(f"ECE tensor shape: {X_ece_torch.shape}")

# --- 3. Create a Custom PyTorch Dataset ---
class HybridDataset(Dataset):
    def __init__(self, images, ece_features, labels):
        self.images = images
        self.ece_features = ece_features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.images[idx], self.ece_features[idx], self.labels[idx]

# --- 4. Define the Hybrid Model Architecture in PyTorch ---
class HybridAnalyzer(nn.Module):
    def __init__(self):
        super(HybridAnalyzer, self).__init__()
        
        # Path 1: CNN (ResNet50)
        self.cnn_base = models.resnet50(weights='IMAGENET1K_V1')
        # Freeze all layers
        for param in self.cnn_base.parameters():
            param.requires_grad = False
        # Replace the final layer
        num_ftrs = self.cnn_base.fc.in_features
        self.cnn_base.fc = nn.Linear(num_ftrs, 64) # Output 64 features
        
        # Path 2: ECE Features
        self.ece_path = nn.Sequential(
            nn.Linear(5, 16), # 5 ECE features in
            nn.ReLU()
        )
        
        # Fusion Head
        self.fusion_head = nn.Sequential(
            nn.Linear(64 + 16, 32), # 64 from CNN + 16 from ECE
            nn.ReLU(),
            nn.Linear(32, 1), # Final output
            nn.Sigmoid()
        )

    def forward(self, x_image, x_ece):
        x1 = self.cnn_base(x_image)   # Get CNN features
        x2 = self.ece_path(x_ece)     # Get ECE features
        x = torch.cat((x1, x2), dim=1) # Concatenate features
        output = self.fusion_head(x)
        return output

# --- 5. Split Data and Create DataLoaders ---
# We split the *indices* to keep data aligned
indices = list(range(len(y_torch)))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42, stratify=y_np)

train_dataset = HybridDataset(X_images_torch[train_indices], X_ece_torch[train_indices], y_torch[train_indices])
val_dataset = HybridDataset(X_images_torch[val_indices], X_ece_torch[val_indices], y_torch[val_indices])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# --- 6. The Training Loop (This replaces .fit()) ---
print("\n--- Training the Analyzer Model (PyTorch) ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = HybridAnalyzer().to(device)
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    model.train() # Set model to training mode
    train_loss = 0.0
    
    for images, ece, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        images, ece, labels = images.to(device), ece.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images, ece)
        loss = criterion(outputs, labels.unsqueeze(1)) # Add dimension for BCELoss
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # --- Validation ---
    model.eval() # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, ece, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]  "):
            images, ece, labels = images.to(device), ece.to(device), labels.to(device)
            
            outputs = model(images, ece)
            loss = criterion(outputs, labels.unsqueeze(1))
            val_loss += loss.item()
            
            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
            
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, "
          f"Val Accuracy: {val_accuracy:.2f}%")

print("\n--- Analyzer Training Complete! ---")

In [ ]:
# --- Step 4: Save the Final Model ---
analyzer_model_path = "/kaggle/working/analyzer_model.pth"
torch.save(model.state_dict(), analyzer_model_path)

print(f"Analyzer model saved to: {analyzer_model_path}")


ANIMATION

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import nibabel as nib
import numpy as np
import cv2
import os
from tqdm import tqdm
from ultralytics import YOLO
from IPython.display import HTML, display

print("--- Final Step: Creating the 'Smart Analyzer' Animation ---")

# --- 1. Define the Analyzer Model Architecture (Needs to be redefined to load weights) ---
# (This is the same class definition from Step 3)
class HybridAnalyzer(nn.Module):
    def __init__(self):
        super(HybridAnalyzer, self).__init__()
        self.cnn_base = models.resnet50(weights='IMAGENET1K_V1')
        for param in self.cnn_base.parameters():
            param.requires_grad = False
        num_ftrs = self.cnn_base.fc.in_features
        self.cnn_base.fc = nn.Linear(num_ftrs, 64)
        self.ece_path = nn.Sequential(nn.Linear(5, 16), nn.ReLU())
        self.fusion_head = nn.Sequential(
            nn.Linear(64 + 16, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    def forward(self, x_image, x_ece):
        x1 = self.cnn_base(x_image)
        x2 = self.ece_path(x_ece)
        x = torch.cat((x1, x2), dim=1)
        output = self.fusion_head(x)
        return output

# --- 2. Load Your TWO Trained Models ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Stage 1: YOLO "Finder"
yolo_model_path = '/kaggle/working/runs/segment/yolov8_lung_seg2/weights/best.pt'
yolo_model = YOLO(yolo_model_path)

# Load Stage 2: ECE "Analyzer"
analyzer_model_path = "/kaggle/working/analyzer_model.pth"
analyzer_model = HybridAnalyzer().to(device)
analyzer_model.load_state_dict(torch.load(analyzer_model_path, map_location=device))
analyzer_model.eval() # Set to evaluation mode
print("All models loaded successfully.")

# --- 3. Select a Test Scan ---
# (Using the same test path loading from your original script)
if 'test_image_paths' not in locals(): # Check if test_image_paths exists
    TEST_IMAGE_DIR = os.path.join(DATA_DIR, "imagesTs")
    test_image_paths = sorted(glob.glob(os.path.join(TEST_IMAGE_DIR, "*/*.nii")))
    
nii_test_image_path = test_image_paths[random.randint(0, len(test_image_paths) - 1)]
print(f"Visualizing test sample: {os.path.basename(os.path.dirname(nii_test_image_path))}")

img_nii = nib.load(nii_test_image_path)
image_data = img_nii.get_fdata()

# --- 4. Setup Video Writer ---
output_final_video_path = '/kaggle/working/final_analyzer_video.mp4'
h, w, num_slices = image_data.shape
frame_size = (w * 2, h) # Original + Analyzer
fps = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_final_video_path, fourcc, fps, frame_size)

# --- 5. Main Video Loop (Running the Full 2-Stage Pipeline) ---
for i in tqdm(range(num_slices), desc="Creating final animation..."):
    # --- Get Original Slice ---
    original_slice = image_data[:, :, i]
    slice_normalized = cv2.normalize(original_slice, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    slice_bgr = cv2.cvtColor(slice_normalized, cv2.COLOR_GRAY2BGR) # For YOLO
    
    panel_original = slice_bgr.copy()
    cv2.putText(panel_original, "Original Slice", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    panel_analyzer = slice_bgr.copy()
    cv2.putText(panel_analyzer, "ECE Analyzer Result", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # --- Stage 1: Run YOLO "Finder" ---
    results = yolo_model.predict(slice_bgr, verbose=False)

    if results[0].masks is not None:
        for mask_tensor, box in zip(results[0].masks.data, results[0].boxes):
            # Get the bounding box
            x1, y1, x2, y2 = box.xyxy[0].int().cpu().numpy()
            
            # --- Prepare Patch for Analyzer ---
            # Crop the patch from the 3-channel BGR image
            patch_bgr = slice_bgr[y1:y2, x1:x2]
            if patch_bgr.size == 0: continue
            
            # 1. Image Tensor
            patch_resized = cv2.resize(patch_bgr, (128, 128))
            patch_transposed = np.transpose(patch_resized, (2, 0, 1)) # C, H, W
            patch_tensor = torch.tensor(patch_transposed, dtype=torch.float32).unsqueeze(0).to(device) / 255.0
            
            # 2. ECE Tensor
            patch_gray = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2GRAY)
            ece_features = get_ece_features(patch_gray) # From Step 2
            ece_tensor = torch.tensor(ece_features, dtype=torch.float32).unsqueeze(0).to(device)

            # --- Stage 2: Run ECE "Analyzer" ---
            with torch.no_grad():
                prediction = analyzer_model(patch_tensor, ece_tensor)
            
            confidence = prediction.item()
            
            # --- Draw the Final Result ---
            if confidence > 0.5: # It's CANCER
                label = f"CANCER ({confidence*100:.0f}%)"
                color = (0, 0, 255) # Red
            else: # It's HEALTHY (False Positive)
                label = f"HEALTHY ({100-confidence*100:.0f}%)"
                color = (0, 255, 0) # Green
                
            # Draw the box
            cv2.rectangle(panel_analyzer, (x1, y1), (x2, y2), color, 2)
            # Draw the label
            cv2.putText(panel_analyzer, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # --- Combine panels and write to video ---
    combined_frame = np.hstack((panel_original, panel_analyzer))
    video_writer.write(combined_frame)

video_writer.release()
print(f"\nFinal animation saved to {output_final_video_path}")

# --- 6. Display the Final Video ---
video_html = f"""
<video width="1024" controls>
  <source src="{output_final_video_path}" type="video/mp4">
</video>
"""
display(HTML(video_html))